# StarGANv2-VC Demo (VCTK 20 Speakers)

### Utils

In [1]:
%cd ..

/media/users/dimaio/tesi/StarGANv2-EmotionalVC


In [2]:
# load packages
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa

from Utils.ASR.models import ASRCNN
from Utils.JDC.model import JDCNet
from models import Generator, MappingNetwork, StyleEncoder
from dataset_maker.emotion_mapping import emotion_map
%matplotlib inline

/media/users/dimaio/miniconda3/envs/stargan/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Source: http://speech.ee.ntu.edu.tw/~jjery2243542/resource/model/is18/en_speaker_used.txt
# Source: https://github.com/jjery2243542/voice_conversion

emotion_map

emotions_label = [emotion_value for emotion_id, emotion_value in emotion_map.items() if emotion_id != "neutral"]
emotions_text = [emotion_id for emotion_id, emotion_value in emotion_map.items() if emotion_id != "neutral"]
to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def build_model(model_params={}):
    args = Munch(model_params)
    generator = Generator(args.dim_in, args.style_dim, args.max_conv_dim, w_hpf=args.w_hpf, F0_channel=args.F0_channel)
    mapping_network = MappingNetwork(args.latent_dim, args.style_dim, args.num_domains, hidden_dim=args.max_conv_dim)
    style_encoder = StyleEncoder(args.dim_in, args.style_dim, args.num_domains, args.max_conv_dim)
    
    nets_ema = Munch(generator=generator,
                     mapping_network=mapping_network,
                     style_encoder=style_encoder)

    return nets_ema

def compute_style(speaker_dicts):
    reference_embeddings = {}
    for key, (path, speaker) in speaker_dicts.items():
        if path == "":
            label = torch.LongTensor([speaker]).to('cuda:1')
            latent_dim = starganv2.mapping_network.shared[0].in_features
            ref = starganv2.mapping_network(torch.randn(1, latent_dim).to('cuda:1'), label)
        else:
            wave, sr = librosa.load(path, sr=24000)
            audio, index = librosa.effects.trim(wave, top_db=30)
            if sr != 24000:
                wave = librosa.resample(wave, sr, 24000)
            mel_tensor = preprocess(wave).to('cuda:1')

            with torch.no_grad():
                label = torch.LongTensor([speaker])
                ref = starganv2.style_encoder(mel_tensor.unsqueeze(1), label)
        reference_embeddings[key] = (ref, label)
    
    return reference_embeddings

### Load models

In [4]:
# load F0 model

F0_model = JDCNet(num_class=1, seq_len=192)
params = torch.load("Utils/JDC/bst.t7")['net']
F0_model.load_state_dict(params)
_ = F0_model.eval()
F0_model = F0_model.to('cuda:1')

In [5]:
# load vocoder
from parallel_wavegan.utils import load_model
vocoder = load_model("Vocoder/PreTrainedVocoder/checkpoint-400000steps.pkl").to('cuda:1').eval()
vocoder.remove_weight_norm()
_ = vocoder.eval()

In [6]:
# load starganv2

model_path = 'Models/Experiment-2/epoch_00100.pth'

with open('Models/Experiment-2/config.yml') as f:
    starganv2_config = yaml.safe_load(f)
starganv2 = build_model(model_params=starganv2_config["model_params"])
params = torch.load(model_path, map_location='cpu')
params = params['model_ema']
_ = [starganv2[key].load_state_dict(params[key]) for key in starganv2]
_ = [starganv2[key].eval() for key in starganv2]
starganv2.style_encoder = starganv2.style_encoder.to('cuda:1')
starganv2.mapping_network = starganv2.mapping_network.to('cuda:1')
starganv2.generator = starganv2.generator.to('cuda:1')

### Conversion

In [8]:
# load input wave
wav_path = 'Demo/neutral.wav'
audio, source_sr = librosa.load(wav_path, sr=24000)
audio = audio / np.max(np.abs(audio))
audio.dtype = np.float32

#### Convert by style encoder

In [16]:
# with reference, using style encoder
emotion_ref={}
print(emotion_map)
for index,val in emotion_map.items():
    print(index)
    if index=="neutral": continue
    emotion_ref[val] = (f'Demo/emotion_sample/{index}/{index}.wav', val)

reference_embeddings = compute_style(emotion_ref)

{'neutral': 0, 'anger': 1, 'happy': 2, 'sad': 3}
neutral
anger
happy
sad


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cuda:0! (when checking argument for argument weight in method wrapper__cudnn_convolution)

In [ ]:
# conversion 
import time
start = time.time()
    
source = preprocess(audio).to('cuda:1')
keys = []
converted_samples = {}
reconstructed_samples = {}
converted_mels = {}

for key, (ref, _) in reference_embeddings.items():
    with torch.no_grad():
        f0_feat = F0_model.get_feature_GAN(source.unsqueeze(1))
        out = starganv2.generator(source.unsqueeze(1), ref, F0=f0_feat)
        
        c = out.transpose(-1, -2).squeeze().to('cuda:1')
        y_out = vocoder.inference(c)
        y_out = y_out.view(-1).cpu()

        if key not in speaker_dicts or speaker_dicts[key][0] == "":
            recon = None
        else:
            wave, sr = librosa.load(speaker_dicts[key][0], sr=24000)
            mel = preprocess(wave)
            c = mel.transpose(-1, -2).squeeze().to('cuda:1')
            recon = vocoder.inference(c)
            recon = recon.view(-1).cpu().numpy()

    converted_samples[key] = y_out.numpy()
    reconstructed_samples[key] = recon

    converted_mels[key] = out
    
    keys.append(key)
end = time.time()
print('total processing time: %.3f sec' % (end - start) )

import IPython.display as ipd
for key, wave in converted_samples.items():
    print('Converted: %s' % key)
    display(ipd.Audio(wave, rate=24000))
    print('Reference (vocoder): %s' % key)
    if reconstructed_samples[key] is not None:
        display(ipd.Audio(reconstructed_samples[key], rate=24000))

print('Original (vocoder):')
wave, sr = librosa.load(wav_path, sr=24000)
mel = preprocess(wave)
c = mel.transpose(-1, -2).squeeze().to('cuda:1')
with torch.no_grad():
    recon = vocoder.inference(c)
    recon = recon.view(-1).cpu().numpy()
display(ipd.Audio(recon, rate=24000))
print('Original:')
display(ipd.Audio(wav_path, rate=24000))

#### Convert by mapping network

In [ ]:
# no reference, using mapping network
speaker_dicts = {}
selected_speakers = [228,230]
for s in selected_speakers:
    k = s
    speaker_dicts['p' + str(s)] = ('', speakers.index(s))

reference_embeddings = compute_style(speaker_dicts)

In [ ]:
# conversion 
import time
start = time.time()
    
source = preprocess(audio).to('cuda:0')
keys = []
converted_samples = {}
reconstructed_samples = {}
converted_mels = {}

for key, (ref, _) in reference_embeddings.items():
    with torch.no_grad():
        f0_feat = F0_model.get_feature_GAN(source.unsqueeze(1))
        out = starganv2.generator(source.unsqueeze(1), ref, F0=f0_feat)
        
        c = out.transpose(-1, -2).squeeze().to('cuda')
        y_out = vocoder.inference(c)
        y_out = y_out.view(-1).cpu()

        if key not in speaker_dicts or speaker_dicts[key][0] == "":
            recon = None
        else:
            wave, sr = librosa.load(speaker_dicts[key][0], sr=24000)
            mel = preprocess(wave)
            c = mel.transpose(-1, -2).squeeze().to('cuda')
            recon = vocoder.inference(c)
            recon = recon.view(-1).cpu().numpy()

    converted_samples[key] = y_out.numpy()
    reconstructed_samples[key] = recon

    converted_mels[key] = out
    
    keys.append(key)
end = time.time()
print('total processing time: %.3f sec' % (end - start) )

import IPython.display as ipd
for key, wave in converted_samples.items():
    print('Converted: %s' % key)
    display(ipd.Audio(wave, rate=24000))
    print('Reference (vocoder): %s' % key)
    if reconstructed_samples[key] is not None:
        display(ipd.Audio(reconstructed_samples[key], rate=24000))

print('Original (vocoder):')
wave, sr = librosa.load(wav_path, sr=24000)
mel = preprocess(wave)
c = mel.transpose(-1, -2).squeeze().to('cuda')
with torch.no_grad():
    recon = vocoder.inference(c)
    recon = recon.view(-1).cpu().numpy()
display(ipd.Audio(recon, rate=24000))
print('Original:')
display(ipd.Audio(wav_path, rate=24000))